# Ex2: Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
import findspark
findspark.init()

In [3]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

In [4]:
sc =SparkContext()

In [5]:
spark = SparkSession(sc)

## Chuẩn bị, chuẩn hóa dữ liệu, xác định input, output

In [7]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("customer_churn.csv", inferSchema=True,header=True)

In [8]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
print((data.count(), len(data.columns)))

(900, 10)


In [10]:
# Khi in bằng head thì định dạng hiển thị là row (khác với head ở pandas dataframe)
data.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [11]:
data.show(3)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 3 rows



In [12]:
for item in data.head():
    print(item)

Cameron Williams
42.0
11066.8
0
7.22
8.0
2013-08-30 07:00:40
10265 Elizabeth Mission Barkerburgh, AK 89518
Harvey LLC
1


In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [21]:
assembler = VectorAssembler(
    inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
    outputCol='features'
)

In [23]:
data_pre = assembler.transform(data)

In [24]:
data_pre.select('features').show(2, truncate=False)

+----------------------------+
|features                    |
+----------------------------+
|[42.0,11066.8,0.0,7.22,8.0] |
|[41.0,11916.22,0.0,6.5,11.0]|
+----------------------------+
only showing top 2 rows



In [25]:
data_pre.show(2)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+--------------------+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|   Company|Churn|            features|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+--------------------+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|Harvey LLC|    1|[42.0,11066.8,0.0...|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|Wilson PLC|    1|[41.0,11916.22,0....|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+--------------------+
only showing top 2 rows



In [26]:
final_data = data_pre.select('features','Churn')

In [27]:
final_data.count()

900

In [28]:
final_data = final_data.na.drop()
final_data.count()

900

In [29]:
train_data, test_data = final_data.randomSplit([0.8,0.2])

In [30]:
train_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                741|
|   mean|0.16194331983805668|
| stddev|0.36864764107410797|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [31]:
test_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                159|
|   mean|0.18867924528301888|
| stddev| 0.3924898018630745|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [32]:
logistic = LogisticRegression(featuresCol='features',
                         labelCol='Churn',
                         predictionCol='prediction')

In [33]:
logisticModel = logistic.fit(train_data)

In [34]:
test_model = logisticModel.transform(test_data)
test_model.groupBy('Churn', 'prediction').count().show()

+-----+----------+-----+
|Churn|prediction|count|
+-----+----------+-----+
|    1|       0.0|   14|
|    0|       0.0|  127|
|    1|       1.0|   16|
|    0|       1.0|    2|
+-----+----------+-----+



In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,  BinaryClassificationEvaluator

In [38]:
binary_evaluator = BinaryClassificationEvaluator(labelCol='Churn')
auc = binary_evaluator.evaluate(test_model,
                               {multi_evaluator.metricName: 'areaUnderROC'})

In [39]:
auc

0.9638242894056847

In [40]:
logisticModel.save('logisticModel_customer_churn')

In [41]:
from pyspark.ml.classification import LogisticRegressionModel
logisticModel2 = LogisticRegressionModel.load('logisticModel_customer_churn')

In [42]:
unlabeled_data = spark.read.csv("new_customers.csv", inferSchema=True,header=True)

In [43]:
unlabeled_data.show(5)

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [46]:
unlabeled_data_pre = assembler.transform(unlabeled_data)

In [47]:
unlabeled_data_pre.show(5)

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [48]:
predictions = logisticModel2.transform(unlabeled_data_pre)

In [49]:
predictions[['features','probability','prediction']].show(5,False)

+----------------------------+------------------------------------------+----------+
|features                    |probability                               |prediction|
+----------------------------+------------------------------------------+----------+
|[37.0,9935.53,1.0,7.71,8.0] |[0.8988830559180624,0.10111694408193761]  |0.0       |
|[23.0,7526.94,1.0,9.28,15.0]|[0.0031487536209678967,0.9968512463790321]|1.0       |
|[65.0,100.0,1.0,1.0,15.0]   |[0.032674022592793416,0.9673259774072066] |1.0       |
|[32.0,6487.5,0.0,9.4,14.0]  |[0.009479586019789035,0.990520413980211]  |1.0       |
|[32.0,13147.71,1.0,10.0,8.0]|[0.7593764742445311,0.24062352575546886]  |0.0       |
+----------------------------+------------------------------------------+----------+
only showing top 5 rows

